In [11]:
import random
import numpy as np
from itertools import combinations
# from bracket import bracket
from tkinter import *
from tkinter import ttk
# import pandas as pd
from tabulate import tabulate
from IPython.display import clear_output
import statistics
import math
import pandas as pd
from openpyxl import load_workbook

In [2]:
#test set
def test(num_players):
  lis=[]
  for i in range(num_players):
    lis.append("P" + str(i+1))
  return lis

In [5]:
# print(int(12/2)%3)
# print(round(7/2))


# index=random.randint(0,len(players)-2)
# print(index)
# del players[0]
# print(players)
# players[index:index+2] = reversed(players[index:index+2])
# print(players)

# print(random.sample(range(0,12),2))
# t=np.zeros((len(players),len(players)))
# t[0][1]=1
# print(t)
#number from 1 to 3
players=test(12)
cycle=1
rem=int(len(players)/2)%3
played_singles=set()
rankings=dict()
player_seed=dict()

#determining the round format. Will be in cycles of 3 rounds.
#for 2/3 rounds, this will be the amount of doubles (all 3 if rem=0)
doubles_main=round(len(players)/6)
singles_main=int((len(players)-4*doubles_main)/2)
#setting number of singles and doubles for each game
round_format=[[doubles_main,singles_main] for i in range(2)]
round_format.append([int(len(players)/2-doubles_main*2),int(len(players)/2-singles_main*2)])

# print(matchup_list(round_format[0], played_singles, players))

In [6]:
def update_rankings():
    return list(dict(sorted(rankings.items(), key=lambda item: (item[1],player_seed[item[0]]),reverse=True)).keys())

def update_rankings_dict():
    return dict(sorted(rankings.items(), key=lambda item: (item[1],player_seed[item[0]]),reverse=True))

def players_by_rank():
    return sorted(truerankings.items(), key=lambda item: item[1],reverse=True)

#make a swap with probability 1-p. Keep swapping until we get outcome p.
def list_noise(lis,p):
    while random.uniform(0,1) > p:
        index=random.randint(0,len(lis)-2)
        lis[index:index+2] = reversed(lis[index:index+2])
    return lis

def game_result(t1,t2,score1,score2,scale=1,win_mult=1.25):
    if type(t1)==str: t1=[t1]
    if type(t2)==str: t2=[t2]
    #half points for doubles
    score1=int(score1)/len(t1)
    score2=int(score2)/len(t2)
    #bonus for winning
    if score1>score2: score1*=win_mult
    if score1<score2: score2*=win_mult
    avt1=statistics.mean([rankings[t] for t in t1])
    avt2=statistics.mean([rankings[t] for t in t2])
    diff=scale*(score1*avt2/avt1-score2*avt1/avt2)

    winner={"T":t1,"S":score1,"A":avt1}
    loser={"T":t2,"S":score2,"A":avt2}
    if diff<0: 
        temp=winner
        winner=loser
        loser=temp

    diff=abs(diff)
    points_to_dist=statistics.mean([min(diff,rankings[t]-10) for t in loser["T"]])
    for t in winner["T"]: rankings[t] += points_to_dist
    for t in loser["T"]: rankings[t] = max(10, rankings[t]-diff)
    # for t in t1: rankings[t] = max(10,rankings[t]+score1*scale*avt2/avt1-score2*scale*avt1/avt2)
    # for t in t2: rankings[t] = max(10,rankings[t]+score2*scale*avt1/avt2-score1*scale*avt2/avt1)

#returns the matchups for the round. Input list of integers, and hashset of players who have done singles.
def matchup_list(round_format, played_singles, players):
    dub=round_format[0]
    sin=round_format[1]

    ppl=players.copy()
    # random.shuffle(ppl)
    # list_noise(ppl,1)
    # if random.randint(1,2)==1:list_noise(ppl,1/len(ppl))
    # else:random.shuffle(ppl)
    list_noise(ppl,1/len(ppl))
    if pr: print(ppl)
    can_single=[p for p in ppl if not p in played_singles]
    # print(random.sample(can_single,2))

    #create singles matches
    singles_ind=sorted(random.sample(range(0,int(len(can_single))),2*sin))
    # print(singles_ind)
    singles_players=[]
    doubles_players=[p for p in played_singles]
    for i in range(len(can_single)):
        if i in singles_ind:
            singles_players.append(can_single[i])
            played_singles.add(can_single[i])
        else:
            doubles_players.append(can_single[i])
    # print(singles_players)
    # print(doubles_players)
    singles_players=[can_single[i] for i in singles_ind]#CHECK THE LAST THREE LINES
    # print(singles_players)
    singles_matches=[[singles_players[2*i],singles_players[2*i+1]] for i in range(int(len(singles_players)/2))]
    for p in singles_matches: random.shuffle(p)
    # print(singles_matches)
    matchups=[[[m[0]],[m[1]]] for m in singles_matches]
    # print("matchups" + str(matchups))

    #create doubles matchups
    doubles_players=[p for p in ppl if p in doubles_players]
    skill_pairs=[[doubles_players[2*i],doubles_players[2*i+1]] for i in range(int(len(doubles_players)/2))]
    if pr: print("pairs: " + str(skill_pairs))
    for p in skill_pairs: random.shuffle(p)
    random.shuffle(skill_pairs)
    doubles_matches=[[[skill_pairs[2*i][0],skill_pairs[2*i+1][0]],
                      [skill_pairs[2*i][1],skill_pairs[2*i+1][1]]] for i in range(int(len(skill_pairs)/2))]
    # print(doubles_matches)
    for d in doubles_matches:matchups.append(d)
    random.shuffle(matchups)
    return matchups

def teams():
    teams=[]
    ordered=update_rankings()
    for i in range(int(len(ordered)/2)):
        # bracket.append(ordr[i])
        teams.append([ordered[i],ordered[len(players)-i-1]])
    return sorted(teams,key=lambda t:sum([rankings[i] for i in t]),reverse=True)

# def random_round():
#     return True
def set_tournament(players):
    # skills=sorted([max(-90*np.log(1-random.uniform(0,1)),random.uniform(0,100)) for i in range(len(players))],reverse=True)
    count=0
    for p in players:
        player_seed[p]=random.uniform(0,1)
        rankings[p]=100
        # truerankings[p]=skills[count]
        count+=1
    # players_sorted=list(dict(sorted(rankings.items(), key=lambda item: (item[1],player_seed[item[0]]),reverse=True)).keys())

def sim_tournament(rounds, players):
    skill=sorted(np.random.exponential(100,len(players)),reverse=True)
    truerankings=dict()
    count=0
    for p in players:
        truerankings[p]=skill[count]
        count+=1
    #number from 1 to 3
    cycle=1
    rem=int(len(players)/2)%3
    played_singles=set()

    #determining the round format. Will be in cycles of 3 rounds.
    #for 2/3 rounds, this will be the amount of doubles (all 3 if rem=0)
    doubles_main=round(len(players)/6)
    singles_main=int((len(players)-4*doubles_main)/2)
    #setting number of singles and doubles for each game
    round_format=[[doubles_main,singles_main] for i in range(2)]
    round_format.append([int(len(players)/2-doubles_main*2),int(len(players)/2-singles_main*2)])

    ordered=update_rankings()
    if pr: print("ranks " + str(update_rankings_dict()))

    cycles=math.ceil(rounds/3)
    rem_rounds=rounds%3
    for i in range(cycles+1):
        played_singles=set()
        random.shuffle(round_format)
        for j in range(3):
            if 3*i+j>=rounds:
                teams=[]
                for i in range(int(len(players)/2)):
                    # bracket.append(ordr[i])
                    teams.append([ordered[i],ordered[len(players)-i-1]])
                if pr: print("Teams: " + str(sorted(teams,key=lambda t:sum([rankings[i] for i in t]),reverse=True)))
                return
            # played_singles=set()
            # round_form=[0,int(len(players)/2)]
            round_form=round_format[j]
            matches=matchup_list(round_form,played_singles,ordered)
            if pr: print("Singles (" + str(3*i+j)+") "+str(played_singles))
            for m in matches:
                t1=sum(truerankings[p] for p in m[0])
                t2=sum(truerankings[p] for p in m[1])
                prob_of_g=t1/(t1+t2)
                sc=[0,0]
                while max(sc)<3:
                    if random.uniform(0,1)<prob_of_g: sc[0]+=1
                    else: sc[1]+=1
                game_result(m[0],m[1],sc[0],sc[1],scale=1,win_mult=1.25)
                if pr: print(str(m[0]) + " " + str(sc[0]) + "-" + str(sc[1]) + " " + str(m[1]))
            ordered=update_rankings()
            if pr: print("ranks " + str(update_rankings_dict()))
            
# set_tournament(players)
# sim_tournament(rounds)

    # print(round_format[0])
    # matchup_list(round_format[0],played_singles)
    #dict(sorted(rankings.items(), key=lambda item: (item[1],random.uniform(0,1)),reverse=True))
    
n=0
pr=True if n==1 else False

results=np.zeros((len(players),len(players)))
in_playoff=np.zeros(len(players))
rounds=max(math.floor(30/(len(players)/3)),3)
for i in range(n):
    set_tournament(players)
    sim_tournament(rounds, players)
    players_sorted=list(dict(sorted(rankings.items(), key=lambda item: (item[1],player_seed[item[0]]),reverse=True)).keys())
    for p in range(len(players_sorted)):
        player_num=int(players_sorted[p][1:])-1
        results[player_num][p]+=1/n
        if p<len(players)/2: in_playoff[player_num]+=1/n

if pr:print(results)
if pr:print(in_playoff)

score=0
for i in range(len(results)):
    for j in range(len(results[i])):
        score+=results[i][j]*abs(i-j) ** 2
        
if pr:print(score)

In [2]:
# player_file = pd.ExcelFile("Players.xlsx")

# dfs = {sheet_name: xl_file.parse(sheet_name) 
#           for sheet_name in xl_file.sheet_names}
# print(dfs)

df = pd.read_excel("Players.xlsx", usecols="A:D")
df_playing=df[df["Status"]=="x"]
players=list(df_playing["Player"])
print(df)

   Player ID Player              Name Status
0          1   Clay    Clayton Allard      x
1          2  David  David Barkemeyer    NaN


In [7]:
#players to play

players=["Clay","Derek","David","Sidhant","Nil","Jeremy","Nehal","Ana","Aaditya","Catherine","Jasper","Sana","Mahdi","Kusai"]


rounds=max(math.floor(30/(len(players)/3)),3)
# rounds=3
rankings=dict()
truerankings=dict()
player_seed=dict()
set_tournament(players)

#To avoid annoying prints
pr=False

#number from 1 to 3
curr_round=0
rem=int(len(players)/2)%3
played_singles=set()

#determining the round format. Will be in cycles of 3 rounds.
#for 2/3 rounds, this will be the amount of doubles (all 3 if rem=0)
doubles_main=round(len(players)/6)
singles_main=int((len(players)-4*doubles_main)/2)
#setting number of singles and doubles for each game
round_format=[[doubles_main,singles_main] for i in range(2)]
round_format.append([int(len(players)/2-doubles_main*2),int(len(players)/2-singles_main*2)])
random.shuffle(round_format)

ordered=update_rankings()
if pr: print("ranks " + str(update_rankings_dict()))

cycles=math.ceil(rounds/3)
rem_rounds=rounds%3
matches=[matchup_list(round_format[0], played_singles, ordered)]
scores=[[["",""] for i in range(len(matches[curr_round]))]]
all_matches=[]

In [9]:
#Setting up GUI for matchups.
root = Tk()
root.title("Foosball Tournament (Round " + str(curr_round+1) + ")")
root.geometry("1280x720")
pr=False
show_ranks=True
# root.attributes('-toolwindow',True)

# root.configure(bg="LightBlue")
# root.iconbitmap('C:\Users\18317\Documents\Foosball\test')
my_entries=[]
    
def naming(team):
    n=str(team[0])
    for i in range(1,len(team)):
        n+= " & " + str(team[i])
    return "(" + n + ")"

def page_setup():
    global main_frame
    main_frame = Frame(root)
    main_frame.pack(fill=BOTH, expand=1)

    my_canvas = Canvas(main_frame)
    my_canvas.pack(side=LEFT, fill=BOTH, expand=1)

    my_scrollbar = ttk.Scrollbar(main_frame, orient=VERTICAL, command=my_canvas.yview)
    my_scrollbar.pack(side=RIGHT, fill=Y)

    my_canvas.configure(yscrollcommand = my_scrollbar.set)
    my_canvas.bind("<Configure>", lambda e: my_canvas.configure(scrollregion=my_canvas.bbox("all")))
    # my_canvas.bind_all("<MouseWheel>", my_canvas.yview_scroll)

    second_frame = Frame(my_canvas)

    my_entries.clear()
    my_canvas.create_window((0,0), window=second_frame, anchor="nw")
    for x in range(len(matches[curr_round])):
        for y in range(2):
            labelNum = Label(second_frame, text=naming(matches[curr_round][x][y])).grid(row=2*x, column=2*y)  
            my_entry=Entry(second_frame,justify='center')
            my_entry.insert(END, str(scores[curr_round][x][y]))
            my_entry.grid(row=2*x+1, column=2*y,sticky='ns',pady=(0,10))
            my_entries.append(my_entry)
    
    if curr_round+1==rounds:
        my_button = Button(second_frame, text="Finish",command=update_scores,height=1,width=10)
    else:
        my_button = Button(second_frame, text="Next Round",command=update_scores,height=1,width=10)
    my_button.grid(row=1,column=3,padx=20)
    return main_frame

#input list to check if any value is not numeric.
def all_numeric(list_of_scores):
    for m in list_of_scores:
        for p in m:
            if not str(p).isnumeric():return False
    return True

def update_scores():
    clear_output(wait=True)
    global curr_round
    global played_singles
    global all_matches
    
    for i in range(int(len(my_entries)/2)):
        val1= my_entries[2*i].get()
        val2= my_entries[2*i+1].get(
        scores[curr_round][i]=[int(val1) if val1.isnumeric() else "",int(val2) if val2.isnumeric() else ""]
    if not all_numeric(scores[curr_round]):
        top= Toplevel(root)
        top.geometry("500x75")
        top.title("Child Window")
        Label(top, text= "Cannot continue to the next round until all scores are reported!", font=(18)).place(x=25,y=25)
        return
    # getting data from GUI     
    # val1=3 if random.randint(0,1)==0 else random.randint(0,2)
    # val2=3 if val1!=3 else random.randint(0,2)
    # scores[i]=[val1,val2]
    # if pr: print("Singles (" + str(curr_round+1)+") "+str(played_singles))  
    
    for s in range(len(scores[curr_round])):
        game_result(matches[curr_round][s][0],matches[curr_round][s][1],scores[curr_round][s][0],scores[curr_round][s][1],scale=1,win_mult=1.25)
        if pr: print(str(matches[curr_round][s][0]) + " " + str(scores[curr_round][s][0]) + "-" + str(scores[curr_round][s][1]) + " " + str(matches[curr_round][s][1]))

    for r in range(len(matches)):
        for m in range(len(matches[r])):
            col_names=["Team 1","Score 1","Score 2","Team 2"]
            all_matches.append([naming(matches[r][m][0]),scores[r][m][0],scores[r][m][1],naming(matches[r][m][1])])
    curr_round+=1
    ordered=update_rankings()
    # if pr: print("ranks " + str(update_rankings_dict())) 
    if show_ranks: 
        table=[[p,ordered[p],rankings[ordered[p]]] for p in range(len(update_rankings_dict()))]
        col_names=["Ranking","Name","Score"]
        print(tabulate(table,headers=col_names, tablefmt="fancy_grid"))
    print(tabulate(all_matches,headers=col_names, tablefmt="fancy_grid"))
    if (curr_round)%3==0:
        played_singles=set()
        random.shuffle(round_format)
    if curr_round==rounds:
        teams=[]
        singles=[]
        for i in range(int(len(players)/2)):
            # bracket.append(ordr[i])
            teams.append([ordered[i],ordered[len(players)-i-1]])
            singles.append([ordered[i]])

        if pr: print("Teams: " + str(sorted(teams,key=lambda t:sum([rankings[i] for i in t]),reverse=True)))
        if pr: print("Singles: " + str(singles))
        root.destroy()
        return
    round_form=round_format[curr_round%3]
    round_matches=matchup_list(round_form,played_singles,ordered)
    if pr: print(round_matches)
    matches.append(round_matches)
    scores.append([(["",""]) for i in range(len(matches[curr_round]))])
    main_frame.destroy()
    page_setup()
    root.title("Foosball Tournament (Round " + str(curr_round+1) + ")")

# update_scores()
if curr_round==rounds:
    print("The tournament is over as the amount of rounds set have been played. To play more rounds, edit the round count in the code.")
    print("Teams: " + str(teams()))
else:
    main_frame=page_setup()
    root.bind('<Return>',lambda event:update_scores())
    root.mainloop()

# my_label = Label(root, text='')
# my_label.grid(row=x+2,column=0,pady=20)

The tournament is over as the amount of rounds set have been played. To play more rounds, edit the round count in the code.
Teams: [['Derek', 'Sana'], ['Sidhant', 'Mahdi'], ['David', 'Aaditya'], ['Clay', 'Jeremy'], ['Kusai', 'Ana'], ['Catherine', 'Jasper'], ['Nil', 'Nehal']]


0


In [12]:
#load excel file
wb = load_workbook(filename="players.xlsx")

with open('Presets.txt') as f:
    lines = f.readlines()
total_players=int(lines[0])
amount_matches=int(lines[1])
 
#open workbook
players_sheet = wb["Players"]
rankings_sheet = wb["Rankings"]
history_sheet = wb["Match History"]
# print(players_sheet.max_column)

for i in range(rankings_sheet.max_row+1,players_sheet.max_row+1):
    rankings_sheet["A"+str(i)]=i-1
    rankings_sheet["B"+str(i)]=players_sheet["A"+str(i)].value
    rankings_sheet["C"+str(i)]=players_sheet["C"+str(i)].value
    rankings_sheet["D"+str(i)]=100
 
#save the file
wb.save(filename="players.xlsx")
    
ranks=pd.read_excel("Players.xlsx", usecols="B:D",sheet_name="Rankings")
ranks=ranks.sort_values(by=["Score"],ascending=False)
ranks.insert(0,"Rank",[i for i in range(1,rankings_sheet.max_row)],False)
print(ranks)

# df = pd.DataFrame([[11, 21, 31], [12, 22, 32], [31, 32, 33]],
#                   index=['one', 'two', 'three'], columns=['a', 'b', 'c'])
 
#save the file


KeyError: 'Worksheet Players does not exist.'

In [49]:
print(update_rankings_dict())
print(update_rankings())
print(teams())

{'Derek': 111.3336892484413, 'Kusai': 110.84652191707418, 'Clay': 109.20539392534964, 'David': 103.01079776219105, 'Nil': 102.34359822876345, 'Sidhant': 99.69927498279893, 'Jeremy': 98.99355886885223, 'Ana': 97.55042611801242, 'Catherine': 96.48035719966067, 'Aaditya': 96.45631369833933, 'Nehal': 96.18878060142681, 'Jasper': 93.69504563124414, 'Mahdi': 92.69144749393452, 'Sana': 91.50479432391131}
['Derek', 'Kusai', 'Clay', 'David', 'Nil', 'Sidhant', 'Jeremy', 'Ana', 'Catherine', 'Aaditya', 'Nehal', 'Jasper', 'Mahdi', 'Sana']
[['Kusai', 'Mahdi'], ['Clay', 'Jasper'], ['Derek', 'Sana'], ['David', 'Nehal'], ['Nil', 'Aaditya'], ['Jeremy', 'Ana'], ['Sidhant', 'Catherine']]
